In [5]:
import tushare as ts
import numpy as np
import wbdata
import pandas as pd
import math
from datetime import datetime, timedelta
import requests
import json
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import os

ts.set_token('ae0addf484ab2b76fe78cdb46b10165b48d31748a48202f9df193951')
pro = ts.pro_api()
start_date = '20000101'
end_date = '20200601'

def add_month(ts, m):
    ts_adj = datetime(ts.year + (ts.month + m // 12), ((ts.month + m % 12)), 1).strftime("%Y-%m-%d")
    return ts_adj

def to_txt(data, name):
    temp = 'w'
#     if ticker == 'AAPL':temp = 'w'
    with open(name, temp) as f:
        for i2, x in enumerate(data[:-1]):
            for i in range(1,len(x)):
                f.write(str(x[i]))
                f.write(',')
                if i == len(x)-1:
                    f.write(str(data[i2+1][1]))
                    
            f.write('\n')

# 财报数据

In [43]:
def get_financialstatement_data(ticker):
    # income
    income_indicator = ['end_date', 'total_revenue', 'total_cogs',\
                 'int_exp', 'oper_exp', 'n_income','ebit','ebitda','basic_eps','operate_profit']
    ds1 = pro.income(ts_code=ticker, start_date=start_date, end_date=end_date, fields=','.join(income_indicator))
    # balance
    balance_indicator = ['end_date','accounts_receiv','acct_payable','inventories','amor_exp','total_cur_assets','intan_assets',\
                        'r_and_d','goodwill','total_assets','total_cur_liab','total_liab']
    ds2 = pro.balancesheet(ts_code=ticker, start_date=start_date, end_date=end_date, fields=','.join(balance_indicator))
    #cash
    ds3 = pro.cashflow(ts_code=ticker, start_date=start_date, end_date=end_date, fields='end_date,c_cash_equ_end_period')
    df = pd.merge(ds1.drop_duplicates(),ds2.drop_duplicates(),how='left',left_on=['end_date'],right_on=['end_date'])
    df = pd.merge(df.drop_duplicates(),ds3.drop_duplicates(),how='left',left_on=['end_date'],right_on=['end_date'])
    df.to_csv('fsa_chinese/' + str(ticker) + '.csv',index=0)
    return df

In [44]:
print(get_financialstatement_data('600000.SH'))

    end_date  basic_eps  total_revenue    total_cogs       int_exp  \
0   20190930      1.620   1.463860e+11  8.861200e+10  1.149540e+11   
1   20190630      1.070   9.759900e+10  5.932900e+10  7.583300e+10   
2   20190331      0.530   5.008400e+10  3.075900e+10  3.825100e+10   
3   20181231      1.850   1.715420e+11  1.061990e+11  1.556440e+11   
4   20180930      1.440   1.271090e+11  7.619100e+10  1.189870e+11   
5   20180630      0.950   8.225600e+10  4.809200e+10  8.132600e+10   
6   20180331      0.460   3.962900e+10  2.252400e+10  4.044400e+10   
7   20171231      1.840   1.686190e+11  9.834400e+10  1.389060e+11   
8   20170930      1.450   1.247680e+11  7.085000e+10  1.014580e+11   
9   20170630      0.970   8.335400e+10  4.737400e+10  6.679000e+10   
10  20170331      0.630   4.236000e+10  2.396300e+10  3.214600e+10   
11  20161231      2.404   1.607920e+11  9.113200e+10  1.066940e+11   
12  20160930      1.873   1.209280e+11  6.741600e+10  7.944800e+10   
13  20160630      1.

In [55]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,list_date')
data = data.values.tolist()
count = 0
for x in data:
    if x[1] <= '20100101':
        get_financialstatement_data(x[0])
        time.sleep(1) # 50/sec
        count += 1
    if count % 100 == 0:print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1600
1600
1600
1600
1600
1600
1600
1600
1600


# Stock Price

In [9]:
def get_stock_price(ticker):
    df = pro.daily(ts_code=ticker, start_date=start_date, end_date=end_date)
    df.to_csv('price_chinese/' + str(ticker) + '.csv',index=0)
    return df

In [42]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,list_date')
data = data.values.tolist()
count = 0
for x in data:
    if x[1] <= '20100101':
        get_stock_price(x[0])
        time.sleep(0.1) # 500/sec
        count += 1
        if count % 100 == 0:print(count)

# GRU_D

In [ ]:
def GRUD_data(raw, data):
    

# GRU

In [6]:
def delta(date, data):
    # 获取对应日期的数据，补缺失值
    dic = {}
    dic_data = dict(zip())
    i = 0
    temp = price[0][1:]
    for x in date:
        while x >= price[i][0]:
            temp = price[i][1:]
            i += 1
        i -= 1
        dic_data[x] = temp
    
    # raw->delta
    for x in range(1,len(date)):
        cur = date[x]
        pre = date[x-1]
        dic[cur] = list(map(lambda x:x[0]/x[1],zip(dic_data[cur],dic_data[pre])))
    return dic

In [ ]:
def GRU_data(date,data):
    

# output

In [ ]:
# stock list
stock_dic = {}
for root, dirs, files in os.walk('price_chinese'):
    for f in files:
        if '.SH' in f:
            stock_dic[f[:-4]] = []
# trading day 
df = pro.trade_cal(exchange='', start_date='20100101', end_date='20191231')
week = set()
df = df.loc[df['is_open']==1]
day = df['cal_date'].values.tolist()
start = datetime.strptime('20100108','%Y%m%d')
i = 0
temp = ''
while start.strftime("%Y%m%d") <= day[-1]:
    b = start.strftime("%Y%m%d")
    if day[i] > b:
        week.add(temp)
        start += timedelta(days=7)
    temp = day[i]
    i += 1
week = sorted(list(week))

# result by day
result = dict(zip(day[1:],[[]]*len(day[1:])))
# stock price
for k in stock_dic.keys():
# for k in ['600136.SH']:
    df = pd.read_csv('price_chinese/' + k + '.csv')
    price = df.loc[:,['trade_date','high','low','close']].values.tolist()
    for i in range(len(price)):
        price[i][0] = str(int(price[i][0]))
    price = sorted(price)
    print(price[-1][0])
    if price[-1][0]<'20191231':
        continue
    r = delta(day,price)
    check = sum([1 for v in r.values() if v == [1.0,1.0,1.0]])/len(day)
    if check >= 0.05:
        continue
    for key,value in r.items():
        result[key].append(value)
    print(result)
print(len(list(stock_dic.keys())),len(result['20100104']))
# to txt



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [44]:
df = pro.daily(ts_code='000016.SH', start_date=start_date, end_date=end_date)
print(df)

Empty DataFrame
Columns: [ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount]
Index: []
